# Divar NLP Workshop - Winter 2021

# Part 2

# Predicting ads Category Based on Descriptions

The objective of this analysis is to train different classifier to predict ads category based on their title and descriptions. The sections of this analysis include:
- split train and test data
- Use tfidf vectorization
  * train a basic classifier with a MultinomialNavieBayes and become familiar with scikit pipelines.
  * explore the results
  * Use other classifications methods by tf-idf and compare the results
  * Choose the best one and improve it
- Use word embeddings technic to classify
    - train a classifier with fasttext


# First Thing First: Load the Data

In [ ]:
%load_ext wurlitzer

In [ ]:
import pandas as pd
import numpy as np

In this part we we want to predict ```cat2``` based on ```preprocessed_text```. So read these columns from your processed data in part 1 of the workshop, (```"outputs/preprocessed_data.parquet"```) and then remove the records that their cat2 is null.

In [ ]:
# TODO:
data = # read data
data = # filter the records that their cat2 is null

Check number of each label in dataset. what are most and least popular cat2? Did a particular point catch your attention? Yeah, It seem that our dataset is unbalanced and we should pay attention to this in future. 

In [ ]:
# TODO:

# Creating Train, validation and Test Datasets

Use any desired method to create *train*, and *test* datasets. Use $60\%$ of whole data at random as *train*, $20\%$ in *validations* and others as *test* datasets. 

In [ ]:
# TODO
train, validation, test = 

You may also want to save your train, validation and test datasets in `.parquet` format under the `./outputs/classification_task/` path for further use.

In [ ]:
!mkdir outputs/classification_task

In [ ]:
# TODO:

Your train, validation and test data is ready now.

Now load saved datasets. and remove the records which their cat2 column is null.

In [ ]:
train = pd.read_parquet('./outputs/classification_task/train.parquet')
validation = pd.read_parquet('./outputs/classification_task/validation.parquet')
test = pd.read_parquet('./outputs/classification_task/test.parquet')


Fill the below variables:

In [ ]:
# TODO:
X_train = 
X_validation = 
X_test = 
y_train = 
y_validation = 
y_test = 

Now create a model by scikit pipiline and CountVectorizer, tfidfTransformer MultinimialNB to predict ```cat2``` based on ```preprocessed_text``` columns. Maybe you should want to read more about sklearn pipeline here:
https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import f1_score


In [ ]:
# TODO: fill ... parts

text_clf = Pipeline([('vect', ...),
                     ('tfidf', ...),
                     ('clf', ...),
                     ])

text_clf.fit(X_train, y_train)



Use predict function of ```text_clf``` to predict ```cat2``` on validation dataset. and then use ```sklearn.metrics.classification_report``` to see some metrics of your classifier.

In [ ]:
# TODO:

preds_validation = 

print(metrics.classification_report(list(y_validation), preds_validation))

print the size of vocabularies by ```len(pipeline.named_steps['count_vect'].vocabulary_.keys())```.

In [ ]:
# TODO:

Let's change the min_df and see the effect of vocabulary size in test performance. For example try with min_df=50.

In [ ]:
# TODO:


How do you interpret this difference? It seems that it is important for search on some important hyperparameters of the models. 

There are different methods and packages for hyperparameter tuning but here let try it by ```sklearn GridSearchCV``` class and become more familiar with sklearn pipelines.

As you remember we had imbalanced datasets and maybe the MultinomialNB fit_prior parameters could be helpful to reduce its impact. First become sure that you know the meaning of fit_prior in NB and then use a GridSearchCV on min_df parameter in range [1, 10 , 50] and fit_prior in range [False, True], use n_splits=3 and scoring='f1_macro'. (what is the difference between f1_macro and f1_weighted?)

Increasing n_jobs can be an option for faster training. But if you still spend a lot of time on training, you can also use a sample of train data in this part.

In [ ]:
from sklearn.model_selection import GridSearchCV, StratifiedKFold
# TODO:
estimator = 
param_grid = 
scoring = 
n_jobs = 
n_splits = 3

pipeline = GridSearchCV(
    estimator=esitmator,
    param_grid=param_grid,
    cv=StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42),
    scoring=scoring,
    verbose=2, 
    n_jobs=n_jobs
)


In [ ]:
pipeline.fit(X_train, y_train)

The outputs of GridSearcCV.fit has interesting information about your different models. Read and use its attributes from:
https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html

What was the best_paramaters?

In [ ]:
# TODO:

For more detailed information see pipeline.cv_results_.

In [ ]:
# TODO:

Select the best estimator and compute classification report on validation dataset and compare it with previous results?

In [ ]:
# TODO: 

Before deep dive into MultinomialNB try to fit some other models. For example try LinearSVC, LogisticRegression and MultinomialNB.

In [ ]:
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression

classifiers = [
    ('naive_bayes', MultinomialNB),
    ('lr', LogisticRegression),
    ('svc', LinearSVC),
    
]

classifier_dict = dict()


for name, clf in classifiers:
    print(name)
    # TODO:
    classifier_dict[name] = 
    ...
    

save the best model to ```./outputs/best_model.pkl```

In [ ]:
import joblib
# TODO:


In [ ]:
best_model = joblib.load('./outputs/best_model.pkl')

We should spend more time on the hyperparameter tuning of these models. But we recommend you to continue this part yourself later. And now let's take a closer look at the best model we have and analyze the sources of it's error.

In the first step, try to plot confusion matrix of the model's predictions using ```plot_confusion_matrix``` function. 

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

%matplotlib inline


def plot_confusion_matrix(y_true, y_pred, classes,
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'

    # Compute confusion matrix
    

    if normalize:
        cm = confusion_matrix(y_true, y_pred, normalize='true')
        print("Normalized confusion matrix")
    else:
        cm = confusion_matrix(y_true, y_pred)
        print('Confusion matrix, without normalization')

#     print(cm)

    fig, ax = plt.subplots(figsize=(16, 12))
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=50, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()

In [ ]:
preds_validation = best_model.predict(X_validation)

In [ ]:
# Plot normalized confusion matrix
# TODO:


Now try to identify areas where the model is not performing well and then see examples of misclassifications in that areas in your validation data.

In [ ]:
# TODO:

Try to write a functions to show most important words for each class. In sklearn pipeline you could access to each steps of pipeline by ```.named_steps['step_name']``` and then you have access to attributes of each step.

In this example after accessing to CountVectorizer step you can access to word names by its function ```get_feature_names()```. And  ```LinearSVC.coef_``` is useful for finding most important coefficients for each class.

In [ ]:
clf = pipeline.best_estimator_.named_steps['clf']

In [ ]:
from matplotlib import pyplot as plt
from sklearn import svm
# TODO:

def f_importances(estimator, k, class_name):
    # it should return list of top k important words and their scores for class_name for example for class_name='animal'
    names = estimator.named_steps['vect'].get_feature_names()
    clf = estimator.named_steps['clf']
    class_index = list(clf.classes_).index(class_name)
    # TODO: create imp and names to return
    
    return imp, names

In [ ]:
imp, names = f_importances(best_model, 40, 'cars')
print(names)

# Part 2 - The End.